In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pandas as pd
import numpy as np
from gtda.time_series import SlidingWindow
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.keras.backend import set_session
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True  
config.log_device_placement = True  
sess2 = tf.compat.v1.Session(config=config)
set_session(sess2) 
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.backend import sigmoid
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.losses import MSE
import tensorflow_datasets as tfds
from tensorflow.keras.models import load_model
import tensorflow_hub as hub

import get_dataset as kws_data
import kws_util
import argparse
from tqdm import tqdm

## Import Dataset

In [ ]:
Flags, unparsed = kws_util.parse_command()
Flags.window_size_ms=40.0
Flags.window_stride_ms=20.0
Flags.batch_size = 1
Flags.data_dir = '/home/nesl/209as_sec/audio_ks/data'
print('We will download data to {:}'.format(Flags.data_dir))
ds_train, ds_test, ds_val = kws_data.get_training_data(Flags)
print("Done getting data")
train_shuffle_buffer_size = 85511
val_shuffle_buffer_size = 10102
test_shuffle_buffer_size = 4890

ds_train = ds_train.shuffle(train_shuffle_buffer_size)
ds_val = ds_val.shuffle(val_shuffle_buffer_size)
ds_test = ds_test.shuffle(test_shuffle_buffer_size)

ds_train = ds_train.map(lambda x, y: (x, tf.one_hot(y, depth=12)))
ds_val = ds_val.map(lambda x, y: (x, tf.one_hot(y, depth=12)))
ds_test = ds_test.map(lambda x, y: (x, tf.one_hot(y, depth=12)))

## Load Model

In [ ]:
model = load_model('/home/nesl/209as_sec/audio_ks/Big Models/CNN/cnn_large.h5')
model.summary()

## Attack

In [ ]:
def fgsm_attack(model, image, label, eps):
    image = tf.cast(image, tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(image)
        pred = model(image)
        loss = MSE(label, pred)
        gradient = tape.gradient(loss, image)
        signedGrad = tf.sign(gradient)
        adversary = (image + (signedGrad * eps)).numpy()
        return adversary

In [ ]:
eps = [0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.5,2.0,5.0,10.0,15.0,20.0,30.0,40.0,50.0]
accu_num = []
for item in eps:
    countadv = 0
    for audio, label in tqdm(ds_test):
        audioLabel = np.array(label).argmax()
        audioPred = model.predict(audio)
        audioPred = audioPred.argmax()
        adversary = fgsm_attack(model,audio, label, eps=item)
        pred = model.predict(adversary)
        adversaryPred = pred[0].argmax()
        if audioPred == adversaryPred:
            countadv += 1

    print("Adversarial accuracy : ", countadv / len(ds_test))
    accu_num.append(countadv / len(ds_test))

In [ ]:
accu_num